In [36]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [48]:
def rle_decode(rle_mask, shape):
    """
    Args:
        rle_mask: run-length as string formated (start length)
        shape: (height, width) of array to return
    Returns:
        numpy array, 1 - mask, 0 - background
    """
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)


def decode_prediction(rle_masks, shape):
    """Decode RLE masks to raster format, 
    where each instance has its unique value
    
    Args:
        rle_masks: list of rle-encoded masks
        shape (tuple): shape of output decoded image, (height, width)
    
    Returns:
        nd.array: np.uint64
    """
    
    output = np.zeros(shape, dtype=np.uint64)
    
    for i, rle_maks in enumerate(tqdm(rle_masks), 1): 
        output += (rle_decode(rle_maks, shape) * i)
        
    return output

In [62]:
# Precision helper function
def precision_at(threshold, iou):
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1   # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    tp, fp, fn = np.sum(true_positives), np.sum(false_positives), np.sum(false_negatives)
    return tp, fp, fn

def f_score(gt, pr, beta=1, verbose=1):
    """
    Calculate instance-wise F-score in range(0.5, 1, 0.05)
    
    Source:
        https://github.com/selimsef/dsb2018_topcoders/blob/master/selim/metric.py
        
    Args:
        gt: ground truth instances mask (each instance has its own unique value)
        pr: predicted instances mask (each instance has its own unique value)
        beta: F-score beta coeffitient
        verbose: verbosity level
        
    Returns:
        score: float
    """
    
    labels = gt
    y_pred = pr
    print_fn = lambda x: print(x) if verbose else None
    
    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred)) 
    
    # Compute intersection between all objects
    intersection = np.histogram2d(labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects))[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins = true_objects)[0]
    area_pred = np.histogram(y_pred, bins = pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection 
    
    # Exclude background from the analysis
    intersection = intersection[1:,1:]
    union = union[1:,1:]
    union[union == 0] = 1e-9

    # Compute the intersection over union
    iou = intersection / union   

    # Loop over IoU thresholds
    prec = []
    print_fn("Thresh\tTP\tFP\tFN\tPrec.")
    for t in np.arange(0.5, 1.0, 0.05):
        tp, fp, fn = precision_at(t, iou)
        p = (1 + beta**2) * tp / ((1 + beta**2) * tp + fp + beta**2 * fn)
        print_fn("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tp, fp, fn, p))
        prec.append(p)
    print_fn("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))
    
    return np.mean(prec)

# Evaluate results

In [59]:
# shapes of source images
shapes = {
    'ZRYNEEUSVQ213QTY.png': (7736, 6530),
    'P2MLF2MV9K9XIYUI.png': (7735, 6529),
    'KHSF2T5PXCKI0978.png': (7734, 6528),
}

ids = list(shapes.keys())

# read data frames with predictions and groun_truth
gt_df = pd.read_csv('../../data/submissions/sub-private-1a.csv')
pr_df = pd.read_csv('../../data/submissions/sub-private-2a.csv')

In [60]:
scores = []

for image_id in ids:
    
    # extract rle masks from dataframes
    gt_rle_masks = gt_df[gt_df.ImageId == image_id].EncodedPixels.values
    pr_rle_masks = pr_df[pr_df.ImageId == image_id].EncodedPixels.values

    # decoding RLE
    gt_mask = decode_prediction(gt_rle_masks, shapes[image_id])
    pr_mask = decode_prediction(pr_rle_masks, shapes[image_id])
    
    # calculate score
    score = f_score(gt_mask, pr_mask, beta=2, verbose=0)
    scores.append(score)

100%|██████████| 757/757 [01:23<00:00,  9.10it/s]


In [61]:
print('F2-score: {:.3}'.format(np.mean(scores)))

F2-score: 0.903
